In [1]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Set up Selenium WebDriver
# Specify the path to chromedriver.exe
driver_path = 'C:/Users/Khushi/Documents/Cooltrack/edgedriver_win64/msedgedriver.exe'
driver = webdriver.Edge(executable_path=driver_path)# You'll need to download the ChromeDriver executable and provide its path

#page=requests.get("https://magicpin.in/New-Delhi/Paharganj/Restaurant/Eatfit/store/61a193/delivery/")

In [3]:
# Navigate to the webpage
driver.get('https://magicpin.in/New-Delhi/Paharganj/Restaurant/Eatfit/store/61a193/delivery/') 

In [4]:
# Find all <span> elements within <p> elements with class "listedItemDetails"
span_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.listedItemDetails span'))
)

# Click on each <span> element
for span_element in span_elements:
    span_element.click()

In [5]:
# Get the updated HTML content
updated_html = driver.page_source

# Parse the updated HTML with BeautifulSoup
soup = BeautifulSoup(updated_html, 'html.parser')
# Beautify the HTML content
beautified_html = soup.prettify()

In [6]:
beautified_html

'<html dir="ltr" lang="en-in">\n <head>\n  <title>\n   Save 15% with Delivery Offers on  \n        EatFit, Paharganj, New Delhi  | December 2023\n  </title>\n  <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.beta.min.js">\n  </script>\n  <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript">\n  </script>\n  <script async="" src="https://www.google-analytics.com/analytics.js">\n  </script>\n  <script>\n   window.dataLayerParams = [{\n            "page_type": "merchantpage_delivery",\n            "affiliate": \'customer_web\'\n        }];\n  </script>\n  <link href="https://static.magicpin.com/samara/media/blog/13-04-2023-favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n  <link href="https://static.magicpin.com/samara/media/blog/13-04-2023-favicon.ico" rel="icon" type="image/x-icon"/>\n  <link crossorigin="" href="https://static.magicpin.com/" rel="preconnect"/>\n  <link href="https://static.magicpin.com/" rel="d

In [7]:
# Initialize a dictionary to store the data
data_dict = {}

# Find all elements with class 'categoryListing' or 'subListingsHeader'
catalog_items = soup.find_all(['article'], class_=['categoryListing'])
current_category = None  # Keep track of the current category heading

In [8]:
catalog_items

[<article class="categoryListing" id="Kulcha Burger"><h4 class="categoryHeading"> Kulcha Burger </h4><div><section class="categoryItemHolder"><div class="itemDetails itemWithImage"><span class="itemImageHolder"><img alt="food type icon" class="foodDescIcon overlap err-handled" src="https://static.magicpin.com/samara/static/images/merchant/magicOrder/veg-icon.svg"/><img alt="Butter Paneer Kulcha Burger's image" class="itemImage err-handled lazyloaded" data-src="https://cdn.urbanpiper.com/media/bizmedia/2023/12/10/dcryWLtz-7_1_t.jpg" src="https://cdn.urbanpiper.com/media/bizmedia/2023/12/10/dcryWLtz-7_1_t.jpg"/></span><article class="itemInfo"><p class="itemName"><a class="rel-handled" href="https://magicpin.in/New-Delhi/Paharganj/Restaurant/product/Butter-Paneer-Kulcha-Burger/?itemId=2857672&amp;mid=19401813&amp;utm_source=merchant_catalog&amp;lat=28.6409887&amp;lon=77.2101869" rel="noopener" style="text-decoration: none;" target="_blank">Butter Paneer Kulcha Burger</a></p><p><span clas

In [9]:
List2=[]
# For example, find all elements with class 'itemInfo'
item_infos = soup.find_all('article', class_='itemInfo')

for item_info in item_infos:
    item_name = item_info.find('p', class_='itemName').a.text.strip()
    item_price = item_info.find('span', class_='itemPrice').text.strip()

    # Save item_name and item_price in List2
    List2.append({'Item Name': item_name, 'Item Price': item_price})


In [10]:
List2

[{'Item Name': 'Butter Paneer Kulcha Burger', 'Item Price': '₹249'},
 {'Item Name': 'Veg Keema Kulcha Burger', 'Item Price': '₹239'},
 {'Item Name': 'Peri-Peri Paneer Kulcha Burger', 'Item Price': '₹308'},
 {'Item Name': 'Spicy Dilli Kulcha Burger', 'Item Price': '₹199'},
 {'Item Name': 'Peri-Peri Chicken Kulcha Burger', 'Item Price': '₹319'},
 {'Item Name': 'Butter Chicken Kulcha Burger', 'Item Price': '₹259'},
 {'Item Name': 'Chicken Keema Kulcha Burger', 'Item Price': '₹259'},
 {'Item Name': 'Veg Manchurian Gravy', 'Item Price': '₹198'},
 {'Item Name': 'Hakka Noodles (500g)', 'Item Price': '₹280'},
 {'Item Name': 'Chilli Garlic Paneer', 'Item Price': '₹260'},
 {'Item Name': 'Chicken Manchurian Gravy', 'Item Price': '₹281'},
 {'Item Name': 'Chilli Garlic Chicken', 'Item Price': '₹330'},
 {'Item Name': 'Veg Manchurian Hakka Noodles Bowl', 'Item Price': '₹284'},
 {'Item Name': 'Veg Manchurian Fried Rice Bowl', 'Item Price': '₹268'},
 {'Item Name': 'Chicken Manchurian Schezwan Noodles B

In [11]:
# Count the number of items in the list
item_count = len(List2)
item_count

84

In [12]:
# Specify the CSV file name
csv_file_name = 'output.csv'
# Replace the currency symbol in the 'Item Price' field

for item in List2:
    if '₹' in item['Item Price']:
        item['Item Price'] = 'Rs.' + item['Item Price'][1:]

In [ ]:
def scrape_and_save_data(url, output_csv, proxies):
    # Set up Selenium WebDriver with a random proxy
    proxy = get_random_working_proxy(proxies)
    if not proxy:
        print("No working proxy found.")
        return
    
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": proxy,
        "ftpProxy": proxy,
        "sslProxy": proxy,
        "proxyType": "MANUAL",
    }

    driver_path = 'C:/Users/Khushi/Documents/Cooltrack/edgedriver_win64/msedgedriver.exe'
    driver = webdriver.Edge(executable_path=driver_path)

    try:
        # Navigate to the webpage
        driver.get(url)

        # Find all <span> elements within <p> elements with class "listedItemDetails"
        span_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.listedItemDetails span'))
        )

        # Click on each <span> element
        for span_element in span_elements:
            span_element.click()

        # Get the updated HTML content
        updated_html = driver.page_source

        # Parse the updated HTML with BeautifulSoup
        soup = BeautifulSoup(updated_html, 'html.parser')

        # Initialize a list to store the data
        data_list = []

        # For example, find all elements with class 'itemInfo'
        item_infos = soup.find_all('article', class_='itemInfo')

        for item_info in item_infos:
            item_name = item_info.find('p', class_='itemName').a.text.strip()
            item_price = item_info.find('span', class_='itemPrice').text.strip()

            # Save item_name and item_price in the list
            data_list.append({'Item Name': item_name, 'Item Price': item_price})

        # Replace the currency symbol in the 'Item Price' field
        for item in data_list:
            if '₹' in item['Item Price']:
                item['Item Price'] = 'Rs.' + item['Item Price'][1:]

        # Save the list to a CSV file
        with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Item Name', 'Item Price']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write the header
            writer.writeheader()

            # Write the data
            writer.writerows(data_list)

        print(f'List saved to CSV file: {output_csv}')

    finally:
        # Close the browser window
        driver.quit()

# Specify the list of websites to scrape
websites = [
    {'url': 'https://website1.com', 'output_csv': 'output1.csv'},
    {'url': 'https://website2.com', 'output_csv': 'output2.csv'},
    # Add more websites as needed
]

# Specify the list of proxies
proxies = ['http://proxy1', 'http://proxy2', 'http://proxy3']

# Scrape data for each website with a random proxy
for website in websites:
    scrape_and_save_data(website['url'], website['output_csv'], proxies)

In [13]:
# Specify the path for the CSV file
csv_file_path = 'output.csv'

# Save the list to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Item Name', 'Item Price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the data
    writer.writerows(List2)

print(f'List saved to CSV file: {csv_file_path}')

List saved to CSV file: output.csv


In [ ]:
# Close the browser window
driver.quit()